In [58]:
import sympy as sympy
import numpy as np
import pandas as pd

from IPython.display import display, Math, HTML


A = sympy.Matrix([
    [-1,2],
    [-5,1],
    [-4,-1]
])
b = sympy.Matrix([
    [4],
    [-16],
    [-11]
])
As = sympy.shape(A)
bs = sympy.shape(b)
_, indcols = A.rref()

dimension = As[1]
rows = As[0]
print("Dimension: ", dimension)
print("ROWS: ", rows)
print("Full column rank: ", A.rank())
# display(HTML(A))

# thanks to https://github.com/RomanBoegli/OptimizHelper
if dimension == 2:
    B = [(i, j) for i in range(rows) for j in range(i, rows)]
else:
    B = [(i, j, k) for i in range(rows)
         for j in range(i, rows) for k in range(j, rows)]
B = [sub_list for sub_list in B if len(set(sub_list)) == dimension]
# check for duplicates
uniquerows = []
duplicaterowindexes = []
for r in range(rows):
    checkrow = list(A.row(r)) + list(b.row(r))
    checkrow_abs = [abs(ele) for ele in checkrow]
    if checkrow_abs in uniquerows:
        duplicaterowindexes.append(r)
    else:
        uniquerows.append(checkrow_abs)
results = []
print("Unique rows: ", uniquerows)
print("Duplicate rows: ", duplicaterowindexes)

i = 0
for B_ in B:
    skip = False
    for elem in list(B_):
        if elem in duplicaterowindexes:
            skip = True
            break
    if skip:
        continue
    i += 1
    B_print = tuple([x+1 for x in list(B_)])
    selection = f'B{i}={B_print}'
    if dimension == 2:
        AB_ = sympy.Matrix.vstack(A.row(B_[0]), A.row(B_[1]))
        bB_ = sympy.Matrix.vstack(b.row(B_[0]), b.row(B_[1]))
    else:
        AB_ = sympy.Matrix.vstack(A.row(B_[0]), A.row(B_[1]), A.row(B_[2]))
        bB_ = sympy.Matrix.vstack(b.row(B_[0]), b.row(B_[1]), b.row(B_[2]))
    _, indrows = AB_.T.rref()
    if len(B_) != len(indrows):
        AB_inv_inexist = AB_.det() == 0

        results.append([
            selection,
            np.array(repr(AB_.tolist())),
            'not invertible' if AB_inv_inexist else np.array(
                repr(AB_.inv().tolist())),
            'x', 'x', f'not a basic selection as\nrow(s) {set(B_print) - set([x+1 for x in list(indrows)])} are\nlinearly dependent'
        ])
        continue
    xB_ = AB_.inv() * bB_
    results.append([
        selection,
        np.array(repr(AB_.tolist())),
        np.array(repr(AB_.inv().tolist())),
        np.array(repr(bB_.tolist())),
        np.array(repr(xB_.T.tolist())),
        'if equal to vertex\n  -> feasible\notherwise infeasible'])

display(HTML(pd.DataFrame(results, columns=[
    "possibility*", "A_Bi", "A_Bi^(-1)", "b_Bi", "xBi.T", "conclusion"]).to_html(formatters={'content': lambda k: k.replace('\n', '<br>')}, escape=False)))
# sum up 1 here to get real values
print("Duplicated rows: ", duplicaterowindexes)


Dimension:  2
ROWS:  3
Full column rank:  2
Unique rows:  [[1, 2, 4], [5, 1, 16], [4, 1, 11]]
Duplicate rows:  []


,possibility*,A_Bi,A_Bi^(-1),b_Bi,xBi.T,conclusion
0,"B1=(1, 2)","[[-1, 2], [-5, 1]]","[[1/9, -2/9], [5/9, -1/9]]","[[4], [-16]]","[[4, 4]]",if equal to vertex\n -> feasible\notherwise infeasible
1,"B2=(1, 3)","[[-1, 2], [-4, -1]]","[[-1/9, -2/9], [4/9, -1/9]]","[[4], [-11]]","[[2, 3]]",if equal to vertex\n -> feasible\notherwise infeasible
2,"B3=(2, 3)","[[-5, 1], [-4, -1]]","[[-1/9, -1/9], [4/9, -5/9]]","[[-16], [-11]]","[[3, -1]]",if equal to vertex\n -> feasible\notherwise infeasible


Duplicated rows:  []
